In [1]:
import sys, os
print("Kernel Python:", sys.executable)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
print("PyTorch:", torch.__version__)

from tqdm import tqdm
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print("Current location:", os.getcwd())

from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig


import torch
print(torch.version.cuda)   # shows the CUDA version PyTorch was built against
print(torch.backends.cudnn.version())  # cuDNN version
print(torch.cuda.is_available())

Kernel Python: /work3/s232958/envs/esm3/bin/python
PyTorch: 2.9.1+cu128
Using device: cpu
Current location: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts


/work3/s232958/envs/esm3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


12.8
91002
False


In [2]:
TRAIN_interaction_df = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_train.csv",index_col=0).reset_index(drop=True)
TEST_interaction_df = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_test.csv",index_col=0).reset_index(drop=True)
TRAIN_interaction_df

,seq1,seq2,ID1,ID2,dimer,seq_target_len,seq_binder_len,target_binder_id
0,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,TVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIHAL...,6NZA_0_A,6NZA_0_B,True,461,459,6NZA_0_A_6NZA_0_B
1,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,9JKA_1_B,9JKA_1_C,True,362,362,9JKA_1_B_9JKA_1_C
2,ARMFEMFNLDWKSGGTMKIKGHISEDAESFAINLGCKSSDLALHFN...,ARMFEMFNLDWKSGGTMKIKGHISEDAESFAINLGCKSSDLALHFN...,2YMZ_0_A,2YMZ_0_B,True,130,130,2YMZ_0_A_2YMZ_0_B
3,AVLLLGEVTNGALNRDATAKAVAAVKALGDVTVLCAGASAKAAAEE...,MKVLVPVKRLIDYNVKARVKSDGSGVDLANVKMSMNPFDEIAVEEA...,1EFP_0_A,1EFP_0_B,False,307,246,1EFP_0_A_1EFP_0_B
4,IDWPALRTQARDAMSRAYAPYSGYPVGAAALVDDGRTVTGCNVENA...,IDWPALRTQARDAMSRAYAPYSGYPVGAAALVDDGRTVTGCNVENA...,6KW6_0_A,6KW6_0_B,True,119,119,6KW6_0_A_6KW6_0_B
...,...,...,...,...,...,...,...,...
5891,KLETIDYRAADSAKRFVESLRETGFGVLSNHPIDKELVERIYTEWQ...,KLETIDYRAADSAKRFVESLRETGFGVLSNHPIDKELVERIYTEWQ...,3ON7_2_C,3ON7_2_D,True,263,266,3ON7_2_C_3ON7_2_D
5892,TLKAAILGATGLVGIEYVRMLSNHPYIKPAYLAGKGSVGKPYGEVV...,RTLKAAILGATGLVGIEYVRMLSNHPYIKPAYLAGKGSVGKPYGEV...,4DPL_2_C,4DPL_2_D,True,353,354,4DPL_2_C_4DPL_2_D
5893,MRVGLVGWRGMVGSVLMQRMVEERDFDLIEPVFFSTSQIGVPAPNF...,MRVGLVGWRGMVGSVLMQRMVEERDFDLIEPVFFSTSQIGVPAPNF...,4R5M_0_A,4R5M_0_B,True,369,369,4R5M_0_A_4R5M_0_B
5894,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSSYAISWVRQAPGQGLE...,6O42_0_L,6O42_0_H,False,214,220,6O42_0_L_6O42_0_H


In [3]:
interaction_Dict = {}

for _, row in TRAIN_interaction_df.iterrows():
    key1 = row['ID1'].split("_")[0]+"_"+row['ID1'].split("_")[-1]
    key2 = row['ID2'].split("_")[0]+"_"+row['ID2'].split("_")[-1]
    seq1 = row['seq1']
    seq2 = row['seq2']       
    interaction_Dict[key1] = seq1
    interaction_Dict[key2] = seq2

for _, row in TEST_interaction_df.iterrows():
    key1 = row['ID1'].split("_")[0]+"_"+row['ID1'].split("_")[-1]
    key2 = row['ID2'].split("_")[0]+"_"+row['ID2'].split("_")[-1]
    seq1 = row['seq1']
    seq2 = row['seq2']       
    interaction_Dict[key1] = seq1
    interaction_Dict[key2] = seq2
    
list(interaction_Dict.items())[:2]

[('6NZA_A',
  'MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIHALALTKRAAAKVNEDLGLLSEEKASAIRQAADEVLAGQHDDEFPLAIWQTGSGTQSNMNMNEVLANRASELLGGVRGMERKVHPNDDVNKSQSSNDVFPTAMHVAALLALRKQLIPQLKTLTQTLNEKSRAFADIVKIGRTHLQDATPLTLGQEISGWVAMLEHNLKHIEYSLPHVAELALGGTAVGTGLNTHPEYARRVADELAVITCAPFVTAPNKFEALATCDALVQAHGALKGLAASLMKIANDVRWLASGPRCGIGEISIPENEPGSSIMPGAVNPTQCEALTMLCCQVMGNDVAINMGGASGNFELNVFRPMVIHNFLQSVRLLADGMESFNKHCAVGIEPNRERINQLLNESLMLVTALNTHIGYDKAAEIAKKAHKEGLTLKAAALALGYLSEAEFDSWVRPEQMVG'),
 ('6NZA_B',
  'TVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIHALALTKRAAAKVNEDLGLLSEEKASAIRQAADEVLAGQHDDEFPLAIWQTGSGTQSNMNMNEVLANRASELLGGVRGMERKVHPNDDVNKSQSSNDVFPTAMHVAALLALRKQLIPQLKTLTQTLNEKSRAFADIVKIGRTHLQDATPLTLGQEISGWVAMLEHNLKHIEYSLPHVAELALGGTAVGTGLNTHPEYARRVADELAVITCAPFVTAPNKFEALATCDALVQAHGALKGLAASLMKIANDVRWLASGPRCGIGEISIPENEPGSSIMPGAVNPTQCEALTMLCCQVMGNDVAINMGGASGNFELNVFRPMVIHNFLQSVRLLADGMESFNKHCAVGIEPNRERINQLLNESLMLVTALNTHIGYDKAAEIAKKAHKEGLTLKAAALALGYLSEAEFDSWVRPEQMVG')]

In [4]:
client = ESMC.from_pretrained("esmc_600m", device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

def calculate_ESM_pr_res_embeddings(sequence):
    protein = ESMProtein(sequence=sequence)
    protein_tensor = client.encode(protein)
    logits_output = client.logits(
    protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
    )
    return logits_output.embeddings.detach().cpu().numpy()

Fetching 4 files: 100%|██████████| 4/4 [00:06<00:00,  1.53s/it]


In [ ]:
path_to_output_embeddings = "/work3/s232958/data/PPint_DB/embeddings_esmC"
os.makedirs(path_to_output_embeddings, exist_ok=True)

# helper: convert torch tensor to numpy
def to_numpy(x):
    try:
        return x.detach().cpu().numpy()
    except AttributeError:
        return np.asarray(x)
    
for name, seq in tqdm(interaction_Dict.items(), total=len(interaction_Dict.items()), desc="Embedding targets"):
    emb = calculate_ESM_pr_res_embeddings(seq)
    emb_np = to_numpy(emb)
    out_path = os.path.join(path_to_output_embeddings, f"{name}.npy")
    np.save(out_path, emb_np)
    print(f"Protein {name} embedded and saved to {out_path}")

Embedding targets:   0%|          | 1/14639 [00:08<35:26:33,  8.72s/it]

Protein 6NZA_A embedded and saved to /work3/s232958/data/PPint_DB/embeddings_esmC/6NZA_A.npy


Embedding targets:   0%|          | 2/14639 [00:13<27:07:57,  6.67s/it]

Protein 6NZA_B embedded and saved to /work3/s232958/data/PPint_DB/embeddings_esmC/6NZA_B.npy


Embedding targets:   0%|          | 3/14639 [00:17<22:12:29,  5.46s/it]

...

Protein 6Z3P_H embedded and saved to /work3/s232958/data/PPint_DB/embeddings_esmC/6Z3P_H.npy


Embedding targets:   0%|          | 45/14639 [01:52<7:27:17,  1.84s/it]

Protein 6Z3P_L embedded and saved to /work3/s232958/data/PPint_DB/embeddings_esmC/6Z3P_L.npy


Embedding targets:   0%|          | 46/14639 [01:55<8:12:41,  2.03s/it]

Protein 3FRC_A embedded and saved to /work3/s232958/data/PPint_DB/embeddings_esmC/3FRC_A.npy
